## Policing in Schools
A look at whether school policing can predict graduation rates.

By Onel Abreu, Alexander Roche, Sabrina Sedovic

In [307]:
import pandas as pd

school_data = pd.read_csv("school_data.csv")
school_data.head(10)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0_x,Unnamed: 0.1.1,Year,Dept/Unit Number,Job Title_Chief Safety & Security OFF,Job Title_Flex Team Security Officer,Job Title_Safety And Security Off,Job Title_School Security Officer,...,# of Unique Students Receiving OSS,% of Unique Students Receiving OSS,Average Length of OSS,# of Police Notifications,% of Misconducts Resulting in a Police Notification,Police Notifications per 100 Students,# of Unique Students Receiving Police Notification,% of Unique Students Receiving Police Notification,# of Students Expelled,Expulsions per 100 Students
0,0,0,1,384,2011,24101,0,0,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,2,898,2012,24101,0,0,0,2,...,5.0,1.6,3.40,1.0,9.1,0.32,1.0,0.3,0.0,0.0
2,2,2,3,1417,2013,24101,0,0,0,2,...,12.0,4.0,1.43,2.0,6.9,0.67,2.0,0.7,0.0,0.0
3,3,3,4,1969,2014,24101,0,0,0,2,...,5.0,1.8,1.80,5.0,38.5,1.78,5.0,1.8,0.0,0.0
4,4,4,5,2489,2015,24101,0,0,0,2,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0
5,5,5,10,450,2011,25151,0,0,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,6,11,965,2012,25151,0,0,0,2,...,31.0,3.4,2.00,1.0,1.0,0.11,1.0,0.1,0.0,0.0
7,7,7,12,1489,2013,25151,0,0,0,2,...,41.0,4.4,2.07,0.0,0.0,0.00,0.0,0.0,0.0,0.0
8,8,8,13,2041,2014,25151,0,0,0,2,...,18.0,1.9,2.42,1.0,1.6,0.11,1.0,0.1,0.0,0.0
9,9,9,14,2559,2015,25151,0,0,0,2,...,10.0,1.0,2.91,1.0,2.4,0.10,1.0,0.1,0.0,0.0


In [308]:
school_data = school_data.drop(columns = ["Unnamed: 0", "Unnamed: 0.1", "Unnamed: 0_x", "Unnamed: 0.1.1",
                                          "Dept/Unit Number", "Job Title_Chief Safety & Security OFF",
                                          "Job Title_Flex Team Security Officer",
                                          "Job Title_Career Counseling Manager",
                                          "Job Title_Director of Counseling",
                                          "Job Title_Director, School Counseling & Post Secondary Advising",
                                          "Job Title_HS Counseling Specialist", 
                                          "Job Title_K‐8 Counseling Specialist",
                                          "Job Title_Manager-Secdry Schl Counselng",
                                          "pos_name_0", "Unit_Number", "Notes", "Unnamed: 26", 
                                          "Unit Number", "Unnamed: 0_y", "Unit", "School_x", 
                                          "School Name_y", "School_y", "Total_y", 
                                          "School Year",'Status as of 2014','Status as of 2019'])

### Data Cleaning

In [309]:
indices = school_data[school_data["Expulsions per 100 Students"].isna() & (school_data["Year"] != 2011)].index
school_data.drop(indices, inplace=True)
school_data.drop(school_data.loc[school_data["5YR Grad Rate"] == ' '].index, inplace=True)
school_data.dropna(subset=['5YR Grad Rate'], inplace=True)
school_data.reset_index(inplace=True)
school_data.drop("index", inplace=True, axis=1)
school_data.dropna(axis=0, inplace=True)
school_data['5YR Grad Rate'] =  school_data['5YR Grad Rate'].astype(float)
print(school_data.columns)

Index(['Year', 'Job Title_Safety And Security Off',
       'Job Title_School Security Officer',
       'Job Title_Senior School Security Officer',
       'Job Title_Senior Security Officer', 'Total Security',
       'Total Security_Filled', 'Job Title_Guidance Counselor Assistant',
       'Job Title_School Counselor', 'Total Counseling',
       'Total Counseling_Filled', 'School ID', 'Total_x', 'Bilingual_N',
       'Bilingual_P', 'IEP_N', 'IEP_P', 'Free_Lunch_N', 'Free_Lunch_P',
       'School Name_x', 'Attendance', '5YR Grad Rate', 'White No', 'White Pct',
       'African American No', 'African American Pct',
       'Native American/Alaskan No', 'Native American/Alaskan Pct',
       'Asian/Pacific Islander No', 'Asian/Pacific Islander Pct',
       'Hispanic No', 'Hispanic Pct', 'Multi-Racial No', 'Multi-Racial Pct',
       'Asian No', 'Asian Pct', 'Hawaiian/Pacific Islander No',
       'Hawaiian/Pacific Islander Pct', 'Not Available No',
       'Not Available Pct', '# of Misconducts'

 ### Initial Data Analysis

In [310]:
%load_ext autoreload
%autoreload 2
import yearsplit
import pipeline as pipeline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [311]:
school_data["Year"] = pd.to_datetime(school_data["Year"], format='%Y')

TimeBasedCV = yearsplit.TimeBasedCV()
splits = TimeBasedCV.split(school_data,  date_column="Year")

Train period: 2012-01-01 - 2013-01-01 , Test period 2013-01-01 - 2014-01-01 # train records 74 , # test records 71
Train period: 2013-01-01 - 2014-01-01 , Test period 2014-01-01 - 2015-01-01 # train records 71 , # test records 88


In [315]:
all_train = school_data.loc[splits[0][0] + splits[1][0], :]

test_1 = school_data.loc[splits[0][1], :]
test_2 = school_data.loc[splits[1][1], :]
suspension_cols = ['# of Misconducts', '# of Group 1-2 Misconducts','# of Group 3-4 Misconducts', '# of Group 5-6 Misconducts',
       '# of Suspensions (includes ISS and OSS)','% of Misconducts Resulting in a Suspension\n(includes ISS and OSS)',
       '# of ISS', '% of Misconducts Resulting in an ISS', 'ISS per 100 Students', '# of Unique Students Receiving ISS',
       '% of Unique Students Receiving ISS', 'Average Length of ISS', '# of OSS', 
       '% of Misconducts Resulting in an OSS', 'OSS per 100 Students', '# of Unique Students Receiving OSS',
       '% of Unique Students Receiving OSS', 'Average Length of OSS', '# of Police Notifications',
       '% of Misconducts Resulting in a Police Notification', 'Police Notifications per 100 Students',
       '# of Unique Students Receiving Police Notification','% of Unique Students Receiving Police Notification',
       '# of Students Expelled', 'Expulsions per 100 Students']
school_data.loc[all_train.index, suspension_cols] = all_train.groupby("School ID")[suspension_cols].transform(lambda x: x.fillna(x.median()))

train_1 = school_data.loc[splits[0][0], :]
train_2 = school_data.loc[splits[1][0], :]

In [317]:
new_splits = []
cross_val_sets = [(train_1, test_1), (train_2, test_2)]
for n, sets in enumerate(cross_val_sets):
    tr, te = sets[0], sets[1]
    tr_norm, scaler = pipeline.normalize(tr, outputinc=True, outputcol="5YR Grad Rate")
    te_norm, _ = pipeline.normalize(tr, scaler=scaler, outputinc=True, outputcol="5YR Grad Rate")
    new_splits.append((tr_norm, te_norm))

### Pipeline

In [328]:
from sklearn.model_selection import GridSearchCV

from sklearn import linear_model
from sklearn import metrics
from sklearn.metrics import make_scorer
from random import randint, uniform

models = {'lr' :linear_model.LinearRegression()}
params = {'lr':[{'fit_intercept': x, 'copy_X': y} 
                           for x in (True, False) \
                           for y in (True, False)]}
scoring = {'MSE': make_scorer(metrics.mean_squared_error)}
pipeline.gridsearch(new_splits,
         models, params, '5YR Grad Rate')

Training model: lr | {'fit_intercept': True, 'copy_X': True} 0
Training model: lr | {'fit_intercept': True, 'copy_X': False} 0
Training model: lr | {'fit_intercept': False, 'copy_X': True} 0
Training model: lr | {'fit_intercept': False, 'copy_X': False} 0
Training model: lr | {'fit_intercept': True, 'copy_X': True} 1
Training model: lr | {'fit_intercept': True, 'copy_X': False} 1
Training model: lr | {'fit_intercept': False, 'copy_X': True} 1
Training model: lr | {'fit_intercept': False, 'copy_X': False} 1
Time Elapsed: 0:00:00.082938


{'fit_intercept': True, 'copy_X': True}  \
lr 0                             1.770088e-28   
   1                             1.001920e-28   

      {'fit_intercept': True, 'copy_X': False}  \
lr 0                              1.770088e-28   
   1                              1.001920e-28   

      {'fit_intercept': False, 'copy_X': True}  \
lr 0                              1.556630e-28   
   1                              2.717656e-28   

      {'fit_intercept': False, 'copy_X': False}  
lr 0                               1.556630e-28  
   1                               2.717656e-28

### Evaluation